In [1]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook summarizes the various layers into the reporting units layer in preperation for EEMS modeling. It also describes the various methodological decisions made in order to summarize the layers into reporting units. Note that the fire probability layers have been converted from raster to vector using Raster to Polygon tool in order to accomodate the summarization process.

**Input Data Layers:**
- Reporting Units (created in `2_Reporting_Units.ipynb`, to be uploaded to DB)
- Land Use Categories (created in `3_Land_Use.ipynb`, to be uploaded to DB)
- [Large Fire Probability](https://databasin.org/datasets/760913fb26844afd9e891209a2dd3811/)
- [Fire Ignition Probability](https://databasin.org/datasets/2d9df577e42b4abe8fd495f75d35a3f1/)
- [WUI Intermix & Interface](https://databasin.org/datasets/218e54cf56f94342a46fefcf05c64f9a/)


In [2]:
arcpy.ResetEnvironments()
gdb_name = "Summarize_Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
# arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

**Load the input data layers**

In [3]:
data = ["in_data/Pr_Lg_Fire.shp", "in_data/Pr_Fire_Ign.shp", "in_data/WUI_Intermix.shp", "in_data/WUI_Interface.shp", 
        "out_data/ember/ember_risk.shp", "out_data/lu/LU_merged.shp", "out_data/ru/Reporting_Units_APN.shp",
        "out_data/dense_dev/point_density_0p5mi.shp", "out_data/dense_dev/point_density_3mi.shp", "out_data/dense_dev/point_density_1000ft.shp"]

for v in data:
    path = os.path.join(THIS_FOLDER, "{}".format(v))
    name = v.rsplit('/', 1)[-1].split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)

**Summarize the fire probability layers into the reporting units. Take the mean value of the fire risk layer for the extent of each reporting unit**

In [4]:
arcpy.analysis.SummarizeWithin("Reporting_Units_APN_rpj", "Pr_Lg_Fire_rpj", 
                               "A_RU_1", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.analysis.SummarizeWithin("A_RU_1", "Pr_Fire_Ign_rpj", 
                               "A_RU_2", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_2
1,


**Summarize the WUI intermix and interface areas into the reporting units. Add shape attributes as we will use those to determine if a reporting unit is inside the WUI or not**

In [5]:
arcpy.analysis.SummarizeWithin("A_RU_2", "WUI_Intermix_rpj", 
                               "A_RU_3", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.analysis.SummarizeWithin("A_RU_3", "WUI_Interface_rpj", 
                               "A_RU_4", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_4
1,


**Summarize the land-use categories into the reporting units. Note that we need to add the majority and minority land-use categories as we will only keep the majority designation and use that to determine the reporting unit land-use category**

In [6]:
arcpy.analysis.SummarizeWithin("A_RU_4", "LU_Merged_rpj", 
                               "A_RU_5", "KEEP_ALL", "LU_Cat Max", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", "LU_Cat", "ADD_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_5
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\LU_Cat_Summary


**Replace all reporting unit null values with 0**

In [7]:
fieldObs = arcpy.ListFields("A_RU_5")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames) 

with arcpy.da.UpdateCursor("A_RU_5", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)

del curU

**Establish the WUI field and populate the value based on the shape area attribute of the summarize within command, then delete the original shape attribute field**

In [8]:
arcpy.AddField_management("A_RU_5", "WUI_Interm", "LONG")
arcpy.AddField_management("A_RU_5", "WUI_Interf", "LONG")

arcpy.CalculateField_management("A_RU_5", "WUI_Interm", 
                                'math.ceil(!SUM_Area_SQUAREKILOMETERS!)', "PYTHON3")
arcpy.CalculateField_management("A_RU_5", "WUI_Interf", 
                                'math.ceil(!SUM_Area_SQUAREKILOMETERS_1!)', "PYTHON3")

arcpy.DeleteField_management("A_RU_5", 
                             ['SUM_Area_SQUAREKILOMETERS', 'SUM_Area_SQUAREKILOMETERS_1'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_5'>

**Rename some of the summarization fields and delete unused ones**

In [9]:
arcpy.AlterField_management('A_RU_5', 
                            'MEAN_gridcode', 'Pr_Lg_Fr', 'Pr_Lg_Fr')
arcpy.AlterField_management('A_RU_5', 
                            'MEAN_gridcode_1', 'Pr_Fire_Ig', 'Pr_Fire_Ig')
arcpy.AlterField_management('A_RU_5', 
                            'Majority_LU_Cat', 'Mj_LU_Cat', 'Mj_LU_Cat')
arcpy.DeleteField_management("A_RU_5", 
                             ['SUM_Area_SQUAREKILOMETERS_12', 'MAX_LU_Cat', 'Minority_LU_Cat', 'Join_ID_1', 'Shape_Leng'])

fieldObs = arcpy.ListFields("A_RU_5")  

fieldNames = []  
for field in fieldObs: 
    print(field.name)
    fieldNames.append(field.name)  
del fieldObs

OBJECTID
Shape
Join_ID
apn
Pr_Lg_Fr
Pr_Fire_Ig
Shape_Length
Shape_Area
Mj_LU_Cat
WUI_Interm
WUI_Interf


**Handle ember risk**

In [10]:
arcpy.analysis.SummarizeWithin("A_RU_5", "ember_risk_rpj", 
                               "A_RU_6", "KEEP_ALL", "sa_emb_rsk Max;sd_emb_rsk Max", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_6
1,


In [11]:
arcpy.AlterField_management('A_RU_6', 
                            'MAX_sa_emb_rsk', 'sa_emb_rsk', 'sa_emb_rsk')
arcpy.AlterField_management('A_RU_6', 
                            'MAX_sd_emb_rsk', 'sd_emb_rsk', 'sd_emb_rsk')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_6'>

In [12]:
field_names = [f.name for f in arcpy.ListFields("A_RU_6")]
print(field_names)

['OBJECTID', 'Shape', 'Join_ID', 'apn', 'Pr_Lg_Fr', 'Pr_Fire_Ig', 'Mj_LU_Cat', 'WUI_Interm', 'WUI_Interf', 'Shape_Length', 'Shape_Area', 'sa_emb_rsk', 'sd_emb_rsk']


**Add building density**

In [13]:
arcpy.analysis.SummarizeWithin("A_RU_6", "point_density_0p5mi", 
                               "A_RU_7", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

arcpy.analysis.SummarizeWithin("A_RU_7", "point_density_3mi", 
                               "A_RU_8", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

arcpy.analysis.SummarizeWithin("A_RU_8", "point_density_1000ft", 
                               "A_RU_9", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_9
1,


In [14]:
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode', 'bl_dns_0p5', 'bl_dns_0p5')
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode_1', 'bl_dns_3mi', 'bl_dns_3mi')
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode_12', 'bl_dns_1kf', 'bl_dns_1kf')

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_9'>

In [15]:
field_names = [f.name for f in arcpy.ListFields("A_RU_9")]
print(field_names)

['OBJECTID', 'Shape', 'Join_ID', 'apn', 'Pr_Lg_Fr', 'Pr_Fire_Ig', 'Mj_LU_Cat', 'WUI_Interm', 'WUI_Interf', 'sa_emb_rsk', 'sd_emb_rsk', 'bl_dns_0p5', 'bl_dns_3mi', 'Shape_Length', 'Shape_Area', 'bl_dns_1kf']


**Export the layer for EEMS modeling. This requires exporting the layer as a CSV, as well as a shapefile, so that we can rejoin the EEMS modeling output. This process uses the Geopandas library**

In [16]:
dir_path = os.path.join(THIS_FOLDER, "out_data/eems")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("A_RU_9", 
                                        os.path.join(THIS_FOLDER, "out_data/eems"))

gdf = gpd.read_file(os.path.join(THIS_FOLDER, "out_data/eems/A_RU_9.shp"))

In [17]:
def normalizeCategory(v):
    if v == 1:
        return 0
    elif v == 2:
        return 0.1
    elif v == 3:
        return 0
    elif v == 4:
        return 0
    else:
        return 1.0

gdf['LU_Cat'] = gdf.apply(lambda row: normalizeCategory(row['Mj_LU_Cat']), axis = 1)

In [18]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Join_ID     18180 non-null  int64   
 1   apn         18179 non-null  object  
 2   Pr_Lg_Fr    18180 non-null  float64 
 3   Pr_Fire_Ig  18180 non-null  float64 
 4   Mj_LU_Cat   18180 non-null  object  
 5   WUI_Interm  18180 non-null  int64   
 6   WUI_Interf  18180 non-null  int64   
 7   sa_emb_rsk  18180 non-null  float64 
 8   sd_emb_rsk  18180 non-null  float64 
 9   bl_dns_0p5  18180 non-null  float64 
 10  bl_dns_3mi  18180 non-null  float64 
 11  Shape_Leng  18180 non-null  float64 
 12  Shape_Area  18180 non-null  float64 
 13  bl_dns_1kf  18180 non-null  float64 
 14  geometry    18180 non-null  geometry
 15  LU_Cat      18180 non-null  float64 
dtypes: float64(10), geometry(1), int64(3), object(2)
memory usage: 2.2+ MB


In [19]:
gdf.to_csv("out_data/eems/A_RU_9.csv", index=False)